# Running Distributed Tensorflow on Ec2

## AWS Setup 
### Navigating to the Console
- Go to the ec2 console page by clicking on "Services" (Top left), then EC2 in the dropdown.
    - On this page you'll see statistics about how many "instances" are running, etc.
- Click the instances tab in the left
    - This page shows you all the instances that are running / terminated / stopped
        - Stopping an instance saves the state of the machine (files, etc from previous runs are left unchanged). But the machine is not alive. 
        - Terminating an instance shuts it down, and the machine is basically gone forever.
        - A running instance is currently alive.
               
### Launching your first EC2
- Click the blue "launch instance" button in the top
- Make sure you are in the ("US-WEST oregon") region, in the top left corner
- Select the "Ubuntu" AMI 
    - Note: AMI = Amazon machine instance = a saved machine state (containing installed software, etc)
        - Amazon has a "marketplace" of AMIs which allow launching instances pre-installed with specific software packages. Additionally you can create your own AMI after configuring a machine with installed software and share it.
- Select t2.micro (free tier) and click "Next:Configure..."
- Leave this page unchanged (skip the "Configure Instance Details" page). Click next.
- Leave this page unchanged (skip the "Add Storage" page). Click next.
- Leave this page unchanged (skip the "Tag Instance" page). Click next.
- On "Step 6: Configure Security Group", make sure SSH can be accessed from a source of "Anywhere". Click Review and Launch.
- Click Launch
    - A popup will appear telling you to select a key pair. Since this is our first time launching an instance, select create a new key pair, name it, and download it to a safe location on your machine. On later launches, you should use pre-existing key pairs.
    - Select Launch Instance
    
### SSH'ing into your EC2 machine
- Navigate to the instances tab of the EC2 console.
- After following the steps for launching the EC2 machine, you should see a new entry in the instance page.
- Selecting the checkbox next to it will show details regarding the machine
    - Click the Connect button at the top. It will tell you how to ssh into that instance.
    - Note: the .pem they refer to is the key file you downloaded when launching the EC2.
   
### Distributed Tensorflow 
#### Basic Info
- In distributed tensorflow there are 2 types of machines 
    - Worker machines, which do gradient computation
    - Parameter servers, which hold the model
    - See https://www.tensorflow.org/versions/r0.11/how_tos/distributed/index.html for more info
- Running distributed tensorflow means
    - Running tensorflow individually across multiple machines (E.G: with 10 machines, tensorflow will be running on each and every one of them)
    - How tensorflow knows which machines are workers, and which are parameter servers
        - Providing the private ips of the machines that create the cluster (this is done through command line args)
        - Providing the type of worker that the machine is via an index (this is done through command line args)
        - Example: 
            - ./bazel-bin/inception/imagenet_distributed_train [... other_args here ...] --worker_hosts='172.31.7.97:1234,172.31.14.51:1234,172.31.9.233:1234,172.31.8.86:1234,172.31.7.247:1234' --ps_hosts='172.31.13.165:1234' --task_id=0 --job_name='worker'
            - worker_hosts is a string containing a comma separated list of private ips in the cluster
            - ps_hosts is a string containing a comma separated list of private ips in the cluster
            - job_name is a string either "worker" or "ps" specifying the type of machine
            - task_id is an integer (0 indexed) specifying the index of the machine in either worker_hosts if the machine is a worker, or ps_hosts if the machin is a ps
    - So if you want a cluster of 10 machines to run the inception model, you need to launch 10 instances on EC2, ssh into each one, and run the appropriate command. I have written some scripts (though they are quite ugly) to make this easier. There are probably better solutions to managing this, but I did not seem to find many that were simple and widely used. Definitely let me know if you find any better ways to manage these jobs!
    
### Launching Via Python Script (MNIST ConvNN)
#### Github

Github: https://github.com/agnusmaximus/DistributedMNIST/tree/clean_mnist

Path to script: https://github.com/agnusmaximus/DistributedMNIST/blob/clean_mnist/tools/tf_ec2.py

#### Overview
The posted python script allows users to manage and configure aws ec2 clusters and run distributed Tensorflow on them (it currently works on MNIST convolutional neural network). It works by having a configuration of the settings of the cluster in a python dictionary, and then using that information to launch / manage clusters. 

As described in the Distributed Tensorflow section, there are two different types of machines -- workers (the master is a type of worker) and parameter servers (ps for short). The script allows you to configure the number of EC2 machines assigned to each, as well as what type of machine is assigned to them, as well as AMI.

Additionally, since training a distributed model oftentimes requires evaluating the model on the dataset to gather loss / accuracy data, the script also launches a separate "evaluator" machine whose sole purpose is to evaluate the trained model (loaded through tensorflow checkpoint files) on the data. The evaluator machine constantly checks to see if a new checkpoint file has been generated -- if so, it loads the new model, and evaluates on the data immediately. Output is written to the same shared directory as the checkpoint. 

These checkpoint files are written to a shared file system (through NFS). Worker / PS output logs are also written to this shared file system to allow easy centralized debugging. This means you can "cd" into the shared filesystem from one of the workers and look at all the logs generated by each machine in the cluster. However this also means that AWS needs to be set up to use NFS. 

#### Getting Set Up
- Do the basic AWS setup (like creating key-pair, etc)
- Install the cli to AWS
    - https://aws.amazon.com/cli/
    - http://docs.aws.amazon.com/cli/latest/userguide/cli-chap-getting-set-up.html
- Set up an EFS (elastic file system) for the NFS shared directory
    - Navigate to the EFS page (you can click "services" and search for "efs")
    - Click "Create File System"
    - Select all us-west availability zones and click next
    - Add an appropriate name for the file system (your name would be good, as it would avoid confusion. I didn't do this, but I should have...)
    - Finish by clicking "Create Filesystem"
    
#### Configuration Description
The configuration file is how you describe cluster properties (number of workers, ps's, AMI, etc). Here I  list several of the more used parameters and describe what they are.
- name - Should be a name for the cluster configuration. It will be used to name the NFS shared directory where all the logs for all the workers are stored.
- key_name - Name of the key pair used to interface with AWS. This is required to distinguish machines of one user from another. That way the shutdown command doesn't shut someone else's machines. 
- n_masters - Number of master workers. This should always be 1.
- n_workers - Number of workers (not including master). This should be set to however many workers you want in the cluster.
- n_ps - Number of parameter servers. This should be set to however many parameter servers you want in the cluster.
- n_evaluators - Number of evaluators. Probably should be 1, since only a single evaluator is required to evaluate a newly written model against data.
- method - "reserved" or "spot". Reserved is required for t2 machines (since they don't support spot instances). Spot instances require you to bid on the price of the machines you're after. Use spot to save money.
- region - Should be "us-west-2"
- availability_zone - Depends on region, but for region="us-west-2" should be one of "us-west-2a","us-west-2b","us-west-2c". Sometimes spot instances are much cheaper in different availability zones. It's useful to check the pricing history on the EC2 console for this data.
- master_type,worker_type,ps_type,evaluator_type - machine tier specification (e.g m4.2xlarge).
- image_id - AMI id which specifies the software that is to be pre-installed on the launched machines. This is important because you should choose an AMI with tensorflow (or additionally your data and extra software) installed. ami-2306ba43 is the TensorflowMnistBase AMI, which has the DistributedMNIST repository downloaded, and tensorflow version 0.12.1 installed. (It also has some basic python packages installed like numpy etc). This is a good starting point for creating new AMI's that rely on tensorflow. 
- spot_price - Price limit for each machine for spot requests. Only matters if method="spot"
- path_to_keyfile - Path to your key pair file. Better to use absolute paths...
- nfs_ip_address - Set this to the ip address of the EFS for the particular availability zone that was set up in the "Set Up" section. To access this info, navigate to the EFS page, click your EFS, and it should tell you the specific ip addressesof the EFS for each availability zone. The availability zone of the nfs ip address should match the availability_zone configuration previously specified.
- nfs_mount_point - Path to the shared filesystem. Something like /home/ubuntu/my_nfs_mount_point. In it will contain all the logs for all clusters (distinguished by the cluster name specified earlier).

#### Basic Usage / Commands
- python tf_ec2.py ...
- python tf_ec2.py launch - Launch instances according to configuration.
- python tf_ec2.py shutdown - Shut down EVERYTHING down. All machines, all spot requests, etc.
- python tf_ec2.py run_tf - Assumes the cluster has been launched. Runs tensorflow on the instances. Then prints commands to ssh into the workers. 
- python tf_ec2.py kill_all_python - runs "pkill -9 python" on all machines. Useful for stopping tensorflow.
- python tf_ec2.py clean_launch_and_run - shutdown, then launch, then run_tf

#### Potential Issues
- IMPORTANT: All of the above so far runs the MNIST distributed training code. Tweaking may be required to run on inception, etc.
    - There may be a problem with the evaluation script (which is currently for MNIST), since I set batch size equivalent to the number of data points (yeah that's bad...). This works on MNIST since it's tiny, but won't work with larger data.
    - The current code (specifically for MNIST) automatically downloads data before each run (this is specific to MNIST). If data is large, it may be helpful to download the data beforehand, create a snapshot of the machine, then use the AMI for machines.
    - The current training code does not set up a pipeline to process (e.g: distort) inputs like the inception code. It just uses feed dictionaries.
    - Since the current code is designed for MNIST, it may be necessary to make changes to accomodate your requirements. (Sorry about this, I was not thinking generality / reusability when I wrote this.)
- If you change the cluster configuration (like the number of overall machines in your cluster), you need to relaunch.
- I have not tested managing multiple clusters (the code is designed so that you can do this, but I have never tried it)... So it may not work (Again, sorry!).

#### Help
If any unexpected error happens or if there are any questions, definitely send me an email at agnusmaximus@gmail.com
